In [ ]:
pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import torch
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [33]:
data = load_dataset("imdb")

In [34]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [35]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\vadim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vadim\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [36]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

In [37]:
data = data.map(preprocess_function,remove_columns = ['text'], batched=True,)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [38]:
#split = imdb['unsupervised'].train_test_split(test_size=0.2,stratify_by_column='label')
#train = split['train']
#test = split['test']
train = data['train']
val = data['test']

In [39]:
del data

In [40]:
accuracy = evaluate.load("accuracy")

In [41]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [42]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2,ignore_mismatched_sizes=True
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [44]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\vadim\AppData\Local\Temp\ipykernel_1512\667385646.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [45]:
trainer.train()

  0%|          | 0/3126 [00:00<?, ?it/s]

{'loss': 0.346, 'grad_norm': 9.699145317077637, 'learning_rate': 1.6801023672424827e-05, 'epoch': 0.32}
{'loss': 0.2729, 'grad_norm': 7.842294692993164, 'learning_rate': 1.3602047344849649e-05, 'epoch': 0.64}
{'loss': 0.2565, 'grad_norm': 8.66749382019043, 'learning_rate': 1.0403071017274472e-05, 'epoch': 0.96}


  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.2697429955005646, 'eval_accuracy': 0.89076, 'eval_runtime': 65.1361, 'eval_samples_per_second': 383.812, 'eval_steps_per_second': 23.996, 'epoch': 1.0}
{'loss': 0.1849, 'grad_norm': 9.783353805541992, 'learning_rate': 7.204094689699297e-06, 'epoch': 1.28}
{'loss': 0.1685, 'grad_norm': 11.21981430053711, 'learning_rate': 4.005118362124121e-06, 'epoch': 1.6}
{'loss': 0.1698, 'grad_norm': 13.125876426696777, 'learning_rate': 8.061420345489445e-07, 'epoch': 1.92}


  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.2732647657394409, 'eval_accuracy': 0.91264, 'eval_runtime': 65.2178, 'eval_samples_per_second': 383.331, 'eval_steps_per_second': 23.966, 'epoch': 2.0}
{'train_runtime': 595.496, 'train_samples_per_second': 83.964, 'train_steps_per_second': 5.249, 'train_loss': 0.22997092163417862, 'epoch': 2.0}


TrainOutput(global_step=3126, training_loss=0.22997092163417862, metrics={'train_runtime': 595.496, 'train_samples_per_second': 83.964, 'train_steps_per_second': 5.249, 'total_flos': 3311684966400000.0, 'train_loss': 0.22997092163417862, 'epoch': 2.0})